In [1]:
from urllib.request import urlopen
from urllib.parse import quote_plus as qp #문자 아스키코드로 변환
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import os

In [2]:
#폴더 생성
search = '순이'
if search not in os.listdir(): os.mkdir(search)  #폴더 생성
imgurl = []

In [3]:
#인터넷 창 띄우기
driver = webdriver.Chrome()
url = 'https://pcmap.place.naver.com/restaurant/1336470115/photo?entry=pll&from=map&fromNxList=true&fromPanelNum=1&ts=1653985461161&filterType=%EC%9D%8C%EC%8B%9D#'
driver.get(url)

In [4]:
scroll_num = 10 #스크롤 횟수
scroll_pause_time=1.5 #스크롤 내릴때 이미지 로딩되는 시간 기다림
upscroll=-100
#스크롤 하기
#스크롤은 기다리기 귀찮으면 직접 한 후에 아래 코드 실행해도 됨
#인터넷 환경에 따라 이미지 로딩이 느릴때, 스크롤이 적게 실행될 수 있음
breaker = [] # 업데이트 5회이상 안될시, break


for i in range(scroll_num+1):

    #파싱
    html = driver.page_source
    soup = BeautifulSoup(html)
    naver = soup.find_all(class_='_img')
    for pic in naver:
        imgurl.append(pic['src']) #이미지 주소 가져오기
        imgurl = (list(set(imgurl))) #이미지주소 중복 제거
    
    print('스크롤 {}번 실행, {}개 그림을 가져올 수 있습니다.'.format(i, len(list(set(imgurl)))))
    breaker.append(len(list(set(imgurl))))
    time.sleep(scroll_pause_time)
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script('scrollBy(0,{})'.format(upscroll))
    
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script('scrollBy(0,{})'.format(upscroll))
        if new_height == last_height:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script('scrollBy(0,{})'.format(upscroll))
        else:
            last_height = new_height
            continue
    if len(breaker)>5 and breaker[-5]==breaker[-1]: 
        print('더이상 update가 없습니다.')
        break

스크롤 0번 실행, 30개 그림을 가져올 수 있습니다.
스크롤 1번 실행, 59개 그림을 가져올 수 있습니다.
스크롤 2번 실행, 89개 그림을 가져올 수 있습니다.
스크롤 3번 실행, 119개 그림을 가져올 수 있습니다.
스크롤 4번 실행, 149개 그림을 가져올 수 있습니다.
스크롤 5번 실행, 179개 그림을 가져올 수 있습니다.
스크롤 6번 실행, 209개 그림을 가져올 수 있습니다.
스크롤 7번 실행, 239개 그림을 가져올 수 있습니다.
스크롤 8번 실행, 269개 그림을 가져올 수 있습니다.
스크롤 9번 실행, 299개 그림을 가져올 수 있습니다.
스크롤 10번 실행, 329개 그림을 가져올 수 있습니다.


In [5]:
for i, pic in enumerate(imgurl):
    with urlopen(pic) as f: #날짜 시간 포함하여 이미지 저장
        with open('./'+search+'/'+search+'_'+'{0:05d}'.format(i+1)+'_'+time.strftime('%y%m%d-%H%M')+'.jpg','wb') as h:
            img = f.read()
            h.write(img)
    print('이미지 저장 중 : {}/{}'.format(i+1,len(imgurl)), end='\r', flush=True)